In [6]:
import numpy as np
import pandas as pd


def add_files_to_fat(files, fat_table):
    for _, (file_start, file_size) in files[["start","size"]].iterrows():
        curr_idx = file_start
        for i in range(file_size-1):
            next_idx  = np.random.choice(np.argwhere(fat_table==0).ravel())
            fat_table[curr_idx] = next_idx
            curr_idx = next_idx
        fat_table[curr_idx] = -1
    

def show_fat_table(fat_table, entries_per_row = 4):
    '''Print a 1D Fat table as a 2d Table'''
    fat_table = fat_table.reshape(-1, entries_per_row)
    for i in range(fat_table.shape[0]):
        print(f"{i*4:3}| {('{:4}'*entries_per_row).format(*fat_table[i])}")
        

def get_blocks(start, fat_table):
    curr_idx = start
    blocks  = []
    while curr_idx != -1:
        blocks.append(curr_idx)
        curr_idx = fat_table[curr_idx]
    return blocks


In [8]:
fat_table_size = 16
n_files = 4
file_names = ["foo","moo","bar","snow","fall"][:n_files]
file_starts = np.random.choice(fat_table_size, n_files, replace = False)
file_sizes = np.random.choice(np.arange(1,(fat_table_size//n_files)), n_files)
files = pd.DataFrame([
    {"name":file_name, "start":file_start ,"size": file_size}
    for file_name, file_start, file_size in zip(file_names, file_starts, file_sizes)
]).set_index("name")

fat_table = np.zeros(fat_table_size,dtype=int)
add_files_to_fat(files, fat_table)

file_name = np.random.choice(file_names)
file_size = files["size"][file_name]
file_blocks = get_blocks(files["start"][file_name],fat_table)

print("Consider the below files and starting blocks and the corresponding FAT table given below.")
display(files[["start"]])
print("FAT Table:")
show_fat_table(fat_table)
print(f"What is the number of blocks allocated for the file {file_name}?")

from quiz_utils import collapsible
from IPython.display import HTML
display(HTML(collapsible(f"File size:{file_size}, File blocks: {file_blocks}")))



Consider the below files and starting blocks and the corresponding FAT table given below.


,start
name,
foo,1
moo,2
bar,7
snow,14


FAT Table:
  0|    9   5  -1   0
  4|   11  14   0   4
  8|    0  -1   0  -1
 12|    0   0   0   0
What is the number of blocks allocated for the file bar?
